<center>
<br />
<h1>Code optimization, Numpy, cython</h1>
<h3>Python</h3>
<br />
<h4>2019</h4> </center>

# Matrix

Create Matrix class:

In [90]:
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        return cls([[0] * shape[1] for i in range(shape[0])])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M
    
    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (len(self), len(self[0])))

The simplest implementation of matrix multiplication:

In [6]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    
    assert n_xcols == n_yrows, "Incompatible matrix dimensions"
    
    Z = Matrix.zeros((n_xrows, n_ycols))
    
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

 

Let's check its correctness:

In [7]:
X = Matrix([[1], [2], [3]])
Y = Matrix([[4, 5, 6]])
print(matrix_product(X, Y))
print(matrix_product(Y, X))


[[4, 5, 6], [8, 10, 12], [12, 15, 18]]
[[32]]


How much time does it take?

In [8]:
def setup(shape=(100, 100)):
    X = Matrix.random(shape)
    Y = Matrix.random(shape)
    return X, Y

In [12]:
%%timeit
matrix_product(*setup())

1 loop, best of 3: 839 ms per loop


Woah! Almost an entire second! That's too long. <br>
Let's find out where we waste time, for that we can use line_profiler:

In [14]:
#!pip3 install --user line_profiler

# If failed:
# git clone https://github.com/rkern/line_profiler.git
# find line_profiler -name '*.pyx' -exec cython {} \;
# cd line_profiler && pip3 install . --user 

In [ ]:
%load_ext line_profiler

In [15]:
%lprun -f matrix_product matrix_product(*setup())

```
Timer unit: 1e-06 s

Total time: 2.83166 s
File: <ipython-input-6-eb2b963d14b1>
Function: matrix_product at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def matrix_product(X, Y):
     2         1         11.0     11.0      0.0      n_xrows, n_xcols = X.shape
     3         1          4.0      4.0      0.0      n_yrows, n_ycols = Y.shape
     4                                               
     5         1          1.0      1.0      0.0      assert n_xcols == n_yrows, "Incompatible matrix dimensions"
     6                                               
     7         1        100.0    100.0      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     8                                               
     9       101         89.0      0.9      0.0      for i in range(n_xrows):
    10     10100       8047.0      0.8      0.3          for j in range(n_xcols):
    11   1010000     836152.0      0.8     29.5              for k in range(n_ycols):
    12   1000000    1987249.0      2.0     70.2                  Z[i][k] += X[i][j] * Y[j][k]
    13         1          2.0      2.0      0.0      return Z
```

We see that a lot of time is spent on for-loop and getting indexes. What do you think can be improved?

In [16]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

In [17]:
%%timeit
matrix_product(*setup())

1 loop, best of 3: 815 ms per loop


Let's try to remove some element retrieval by index:

In [18]:
def matrix_product2(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi = X[i]
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += Xi[j] * Y[j][k]
            Z[i][k] = acc
    return Z

In [19]:
%%timeit
matrix_product2(*setup())

1 loop, best of 3: 402 ms per loop


2 times faster! Since it became about 2 times less operations of retrieval by index.

In [20]:
%lprun -f matrix_product2 matrix_product2(*setup())

```
Timer unit: 1e-06 s

Total time: 2.16743 s
File: <ipython-input-18-bcd941f4d272>
Function: matrix_product2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def matrix_product2(X, Y):
     2         1         10.0     10.0      0.0      n_xrows, n_xcols = X.shape
     3         1          3.0      3.0      0.0      n_yrows, n_ycols = Y.shape
     4         1         89.0     89.0      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     5       101         75.0      0.7      0.0      for i in range(n_xrows):
     6       100        129.0      1.3      0.0          Xi = X[i]
     7     10100       8348.0      0.8      0.4          for k in range(n_ycols):
     8     10000       7774.0      0.8      0.4              acc = 0
     9   1010000     807968.0      0.8     37.3              for j in range(n_xcols):
    10   1000000    1331108.0      1.3     61.4                  acc += Xi[j] * Y[j][k]
    11     10000      11925.0      1.2      0.6              Z[i][k] = acc
    12         1          1.0      1.0      0.0      return Z
```

Let's go further and try to be even faster!

In [23]:
def matrix_product3(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()  # :)
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            getX = Xi.__getitem__
            Zi[k] = sum(getX(j) * Ytk[j] for j in range(n_xcols))
    return Z

In [21]:
X = [[1],[2],[3]]
Y = [[4,5,6]]
Z = [[0,0,0],[0,0,0],[0,0,0]]
Yt = [[4],[5],[6]]

for i, (Xi, Zi) in enumerate(zip(X, Z)):
        print((i,Xi,Zi))
        for k, Ytk in enumerate(Yt):
            print((k,Ytk))

(0, [1], [0, 0, 0])
(0, [4])
(1, [5])
(2, [6])
(1, [2], [0, 0, 0])
(0, [4])
(1, [5])
(2, [6])
(2, [3], [0, 0, 0])
(0, [4])
(1, [5])
(2, [6])


In [24]:
%%timeit
matrix_product3(*setup())

1 loop, best of 3: 256 ms per loop


Again almost 2 times faster!

Conclusion: if your code is slow, do not rush to use other frameworks. Often you can speed up your code at times with little manipulation.

# Numpy

Let's try to take our first implementation, but use an array from numpy instead (we'll see how it works later)

In [25]:
import numpy as np

def np_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [26]:
def setup_np(shape=(100, 100)):
    X = np.random.randint(-255, 255, size=shape)
    Y = np.random.randint(-255, 255, size=shape)
    return X, Y

In [27]:
%timeit np_matrix_product(*setup_np())

1 loop, best of 3: 1.2 s per loop


No profit so fat.

In [28]:
X, Y = setup_np()

In [29]:
%timeit np.dot(X, Y)

1000 loops, best of 3: 1.72 ms per loop


It's getting interesting. Good motivation to learn the numpy module.

## Array

In [31]:
a = np.array([4, 1.1, 2, 3])

a[1] = 5

print(a.size, a.shape, a.dtype)
print(a)

4 (4,) float64
[4. 5. 2. 3.]


You can see that the array is effectively stored in memory:

In [32]:
print(a.nbytes)
a = np.array([4,1.1,2])
print(a.nbytes)

32
24


Shape can be changed:

In [33]:
a = np.array([4, 1.1, 2, 3])

a[1] = 5

a.shape = (2, 2)
a

array([[4., 5.],
       [2., 3.]])

An array is of one type. If you create differently typed elements, they all will be cast to the same type.

In [34]:
np.array([4, 1.1, 2, 3, "string"])

array(['4', '1.1', '2', '3', 'string'], dtype='<U32')

## UFuncs

UFuncs (universal functions) are functions that work element by element.

In [35]:
a = range(4)
b = [value + 1 for value in a]
print(b)

[1, 2, 3, 4]


In numpy:

In [36]:
a = np.arange(4)
b = a + 1
print(b)

[1 2 3 4]


The addition of two arrays.

In [37]:
count = int(1e7)
a, b = [random.random() for _ in range(count)], [random.random() for _ in range(count)]

%timeit [i + j for (i, j) in zip(a, b)]

1 loop, best of 3: 1.44 s per loop


In [38]:
a, b = np.array(a), np.array(b)
%timeit a + b

10 loops, best of 3: 67.3 ms per loop


It's because numpy is written in c.

Other UFuncs

In [39]:
def slow():
    a = range(10000)
    return [i ** 2 for i in a]

%timeit slow()

100 loops, best of 3: 4.63 ms per loop


In [41]:
def fast():
    a = np.arange(10000)
    return a ** 2

%timeit fast()

The slowest run took 5.62 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 24.1 µs per loop


There are many other UFuncs: arithmetic operations, bitwise, comparisons, trigonometric, etc.

## Aggregation

Numpy has many built-in aggregation functions

<img src="aggregations.png" width=600px>

In [42]:
data = [random.random() for _ in range(1000000)]
%timeit min(data)

10 loops, best of 3: 38.8 ms per loop


In [43]:
np_data = np.array(data)
%timeit np.min(np_data)

The slowest run took 6.67 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 880 µs per loop


## Indexing

There are all basic slices and indexes. But there are many additional tasty features:

In [45]:
X = np.arange(6).reshape((2, 3))
X

array([[0, 1, 2],
       [3, 4, 5]])

In [46]:
X[1]

array([3, 4, 5])

In [47]:
X[0, 1]

1

In [48]:
z = X[:1, 1:]
z

array([[1, 2]])

Unlike with standard lists, numpy slices are not copies and you can change the initial list.

In [49]:
z[:1, :1] = 100

In [50]:
X

array([[  0, 100,   2],
       [  3,   4,   5]])

### Masking

In [51]:
a = np.arange(8) ** 2
a[a > 8]

array([ 9, 16, 25, 36, 49])

This works because numpy.array supports boolean mask indexing:

In [59]:
a > 8

array([False, False, False,  True,  True,  True,  True,  True])

You can also use an array of indices:

In [55]:
a = np.arange(4) * 2
print(a)
a[[3, 3, 2, 2, 0, 0, 1, 1]]

[0 2 4 6]


array([6, 6, 4, 4, 0, 0, 2, 2])

## Broadcasting
Or automatic casting of dimensions.

You can stack arrays of different sizes where it makes sense:

In [56]:
a = np.arange(3)

In [57]:
b = np.array(((0, 0, 0), (10, 10, 10), (20, 20, 20), (30, 30, 30)))
b

array([[ 0,  0,  0],
       [10, 10, 10],
       [20, 20, 20],
       [30, 30, 30]])

In [58]:
b + a

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

<img src="https://i.stack.imgur.com/JcKv1.png" width=900px/>

But it does not always work as we would like:

In [59]:
X = np.arange(6).reshape((3, 2))
X

array([[0, 1],
       [2, 3],
       [4, 5]])

In [60]:
X + np.array([1, 2, 3])

ValueError: operands could not be broadcast together with shapes (3,2) (3,) 

Of course, we understand what was meant, but numpy did not understand. We will tell him:

In [62]:
Y = np.array((1, 2, 3))[:, np.newaxis]  # reorient array
print(Y.shape)
print(Y)

(3, 1)
[[1]
 [2]
 [3]]


In [63]:
X + Y

array([[1, 2],
       [4, 5],
       [7, 8]])

Conclusion: Loops in python are slow. It is worth using the built-in numpy functions soo the code is more understandable and significant acceleration is gained.

# Cython 
Yet another way to optimize your code

The Cython compiler translates a program written in Python or Cython into more efficient C code

Cython is a Python add-on, supporting C function calls and assignment C language data type variables

Let's see how it works. Let's create a couple of functions in pure python:

In [67]:
def f(x):
    return x**2

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

In [80]:
%%timeit
integrate_f(0, 10, 1000000)

1 loop, best of 3: 507 ms per loop


Enabling Cython magic:

In [ ]:
%load_ext Cython

In [70]:
%%cython -a 
def f(x):
    return x**2

def integrate_f2(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

Cython is trying to determine variable types and due to this speed up. But those lines of code where he could not establish the type are highlighted in yellow. Almost everything is yellow.

In [74]:
%%timeit
integrate_f2(0, 10, 1000000)

1 loop, best of 3: 462 ms per loop


It's a bit faster, but we can do better.

Let's apply static typing:

In [81]:
%%cython -a 
def f(double x):
    return x**2

def integrate_f3(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):  # when i is defined as int the loop is compiled into pure C code
        s += f(a + i * dx)
    return s * dx

In [82]:
%%timeit
integrate_f3(0, 10, 1000000)

10 loops, best of 3: 92 ms per loop


Function calls in python can be expensive. For example, in the last example, there was a recurrent conversion from C double to Python float. 

Let's change the prototype of the function f:

In [85]:
%%cython -a 
cdef double f2(double x):
    return x**2

def integrate_f4(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):  # when i is defined as int the loop is compiled into pure C code
        s += f2(a + i * dx)
    return s * dx

In [88]:
%%timeit
integrate_f4(0, 10, 1000000)

100 loops, best of 3: 9.53 ms per loop


Let's compare with numpy

In [89]:
%%timeit
x = np.linspace(0, 10, 1000000)
(x**2).sum() / 100000

100 loops, best of 3: 14.3 ms per loop


With Cython you can achieve more than with numpy!

## Matrix
Back to our Matrix example

In [91]:
%%cython -a 
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M

    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (int(len(self)), int(len(self[0]))))


def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()
    for i, Xi in enumerate(X):
        for k, Ytk in enumerate(Yt):
            Z[i][k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z


The whole thing is yellow!

In [92]:
shape = (100, 100)

In [93]:
X = Matrix.random(shape)
Y = Matrix.random(shape)

In [94]:
%timeit cy_matrix_product(X, Y)

10 loops, best of 3: 157 ms per loop


It is faster than our Python implementation, but so far it does not reach the numpy version. <br>
Let's try to tell cython what types we use, maybe it will be faster.

In [95]:
X = np.random.randint(-255, 255, size=shape, dtype=np.int64)
Y = np.random.randint(-255, 255, size=shape, dtype=np.int64)

In [96]:
%%cython -a
import numpy as np

def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [97]:
%timeit cy_matrix_product(X, Y)

1 loop, best of 3: 984 ms per loop


Now it's worse!

Cython can't work with Python imports and now it's required to ask python for every little snippet of code.

Let's explain data types for Cython explicitly.

In [98]:
%%cython -a
import numpy as np
cimport numpy as np

def cy_matrix_product(np.ndarray X, np.ndarray Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray Z
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [99]:
%timeit  cy_matrix_product(X, Y)

1 loop, best of 3: 929 ms per loop


It seems that cython now understands more, but it's still slow, since the main parts are still not clear.

Let's add static typing for input values - now our function takes only values of a specified type.

In [103]:
%%cython -a
import numpy as np
cimport numpy as np  # Not every module may be imported like this


cdef cy_matrix_product(np.ndarray[np.int64_t, ndim=2] X,
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [104]:
%timeit cy_matrix_product(X, Y)

1 loop, best of 3: 920 ms per loop


Well, now we’ll add quite a bit of black magic. <br>
Add decorators that check the boundaries of arrays and type overflows.

In [112]:
%%cython -a
import numpy as np

cimport cython
cimport numpy as np

@cython.boundscheck(False)
@cython.overflowcheck(False)
def cy_matrix_product(np.ndarray[np.int64_t, ndim=2] X, 
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):        
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [113]:
%timeit cy_matrix_product(X, Y)

100 loops, best of 3: 3.37 ms per loop


Profit.